## Teil P.2: ETL Pipeline

#### API
https://api.jolpi.ca/ergast/

#### Kennzahlen und Dimensionen
- Durchschnittliche Qualifying-Zeit nach Qualifying Run (Q1, Q2, Q3), (erzielter) Startposition, Saison
- Anzahl der Rennsiege nach Fahrer, Alter, Nationalitäten der Fahrer
- Anzahl technischer Ausfälle durch Motorprobleme nach Fahrer, Team, Saison

### Import Anweisungen

In [2]:
import duckdb as db
import pandas as pd
import numpy as np
import requests
import time

ModuleNotFoundError: No module named 'duckdb'

### Erstellung der Datenbank

In [ ]:
con = db.connect("f1.duckdb")

with open("Installationsskript.sql", "r") as f:
    sql = f.read()
    con.execute(sql)

### Abfrage der Daten

#### Fahrer

In [ ]:
fahrer = []

for season in range(2023, 2026):
    url = f"https://api.jolpi.ca/ergast/f1/{season}/drivers.json"
    
    try:
        resp = requests.get(url)
        data = resp.json()
        drivers = data["MRData"]["DriverTable"]["Drivers"]

        if drivers:
            for driver in drivers:
                fahrer.append({
                    "Fahrer_ID": driver["driverId"],
                    "Name": driver["givenName"],
                    "Nationalität": driver["nationality"],
                    "Geburtsdatum": driver["dateOfBirth"]
                })
            
    except Exception as e:
        print(f"Fehler bei {season}: {e}")

df_fahrer = pd.DataFrame(fahrer)

print(df_fahrer.head())

  Fahrer_ID       Name Nationalität Geburtsdatum
0     albon  Alexander         Thai   1996-03-23
1    alonso   Fernando      Spanish   1981-07-29
2    bottas   Valtteri      Finnish   1989-08-28
3  de_vries       Nyck        Dutch   1995-02-06
4     gasly     Pierre       French   1996-02-07


#### Konstrukteure

In [ ]:
konstrukteure = []

for season in range(2023, 2026):
    url = f"https://api.jolpi.ca/ergast/f1/{season}/constructors.json"
    
    try:
        resp = requests.get(url)
        data = resp.json()
        constructors = data["MRData"]["ConstructorTable"]["Constructors"]

        if constructors:
            for constructor in constructors:
                konstrukteure.append({
                    "Konstrukteur_ID": constructor["constructorId"],
                    "Name": constructor["name"]
                })
    
    except Exception as e:
        print(f"Fehler bei Konstrukteure {season}: {e}")

df_konstrukteure = pd.DataFrame(konstrukteure)

print(df_konstrukteure.head())

  Konstrukteur_ID            Name
0            alfa      Alfa Romeo
1      alphatauri      AlphaTauri
2          alpine  Alpine F1 Team
3    aston_martin    Aston Martin
4         ferrari         Ferrari


#### Rennen

In [ ]:
rennen = []

for season in range(2023, 2026):
    url = f"https://api.jolpi.ca/ergast/f1/{season}.json"
    try:
        response = requests.get(url)
        data = response.json()
        races = data["MRData"]["RaceTable"]["Races"]

        for race in races:
            rennen.append({
                "Rennen_ID": f"{season}_{race['round']}",
                "Saison": season,
                "GP_Name": race["raceName"],
                "Datum": race["date"],
                "Runde": int(race["round"])
            })

    except Exception as e:
        print(f"Fehler bei Saison {season}: {e}")

df_rennen = pd.DataFrame(rennen)

print(df_rennen.head())

  Rennen_ID  Saison                   GP_Name       Datum  Runde
0    2023_1    2023        Bahrain Grand Prix  2023-03-05      1
1    2023_2    2023  Saudi Arabian Grand Prix  2023-03-19      2
2    2023_3    2023     Australian Grand Prix  2023-04-02      3
3    2023_4    2023     Azerbaijan Grand Prix  2023-04-30      4
4    2023_5    2023          Miami Grand Prix  2023-05-07      5


#### Qualifying Results

Die Ergast API hat ein Rate Limit von 4 Requests pro Sekunde. Da wir auf recht große Datenmengen zugreifen müssen, um alle Qualifying- und Rennergebnisse zu erhalten, müssen wir das Rate Limit beachten. Dazu haben wir die folgende Methode geschrieben:

In [ ]:
def safe_request(url, max_retries=3, delay=0.5):
    for attempt in range(max_retries):
        try:
            resp = requests.get(url)
            if resp.status_code == 200:
                return resp.json()
            elif resp.status_code == 429:
                time.sleep(2)
            else:
                print(f"HTTP Fehler {resp.status_code} bei {url}")
                break
        except Exception as e:
            print(f"Fehler beim Abrufen von {url}: {e}")
        time.sleep(delay)
    return None

In [ ]:
results_quali = []

for season in range(2023, 2026):
    season_url = f"https://api.jolpi.ca/ergast/f1/{season}.json"
    season_data = safe_request(season_url)
    time.sleep(0.3)

    if not season_data:
        continue

    races = season_data["MRData"]["RaceTable"]["Races"]

    for race in races:
        rnd = race["round"]
        quali_url = f"https://api.jolpi.ca/ergast/f1/{season}/{rnd}/qualifying.json"
        quali_data = safe_request(quali_url)
        time.sleep(0.3)

        if not quali_data:
            continue

        quali_races = quali_data["MRData"]["RaceTable"]["Races"]
        if quali_races:
            quali = quali_races[0]
            for q in quali["QualifyingResults"]:
                driver = q["Driver"]
                results_quali.append({
                    "Saison": season,
                    "Runde": int(rnd),
                    "Fahrer_ID": driver["driverId"],
                    "Startplatz": int(q["position"]),
                    "Q1": q.get("Q1"),
                    "Q2": q.get("Q2"),
                    "Q3": q.get("Q3")
                })

df_quali = pd.DataFrame(results_quali)
print(df_quali.head())

   Saison  Runde       Fahrer_ID  Startplatz        Q1        Q2        Q3
0    2023      1  max_verstappen           1  1:31.295  1:30.503  1:29.708
1    2023      1           perez           2  1:31.479  1:30.746  1:29.846
2    2023      1         leclerc           3  1:31.094  1:30.282  1:30.000
3    2023      1           sainz           4  1:30.993  1:30.515  1:30.154
4    2023      1          alonso           5  1:31.158  1:30.645  1:30.336


#### Ergebnisse

In [ ]:
ergebnisse = []

for season in range(2023, 2026):
    season_url = f"https://api.jolpi.ca/ergast/f1/{season}.json"
    season_data = safe_request(season_url)
    time.sleep(0.3)

    if not season_data:
        continue

    races = season_data["MRData"]["RaceTable"]["Races"]

    for race in races:
        rnd = race["round"]
        results_url = f"https://api.jolpi.ca/ergast/f1/{season}/{rnd}/results.json"
        data = safe_request(results_url)
        time.sleep(0.3)

        if not data:
            continue

        results_races = data["MRData"]["RaceTable"]["Races"]
        if results_races:
            results_race = results_races[0]
            rennen_id = f"{season}_{rnd}"

            for result in results_race["Results"]:
                ergebnisse.append({
                    "Rennen_ID": rennen_id,
                    "Fahrer_ID": result["Driver"]["driverId"],
                    "Position": int(result["position"]),
                    "Status": result["status"],
                    "Konstrukteur_ID": result["Constructor"]["constructorId"]
                })

df_ergebnisse = pd.DataFrame(ergebnisse)
print(df_ergebnisse.head())


  Rennen_ID       Fahrer_ID  Position    Status Konstrukteur_ID
0    2023_1  max_verstappen         1  Finished        red_bull
1    2023_1           perez         2  Finished        red_bull
2    2023_1          alonso         3  Finished    aston_martin
3    2023_1           sainz         4  Finished         ferrari
4    2023_1        hamilton         5  Finished        mercedes


### Transformation der Daten

- Daten in das Format Date bringen (Momentan String), um Berechnungen für das Alter durchführen zu können
- Die Quali Zeiten in ein Millisekundenformat bringen (Momentan String), um Berechnungen für die Durchschnittliche Zeit durchzuführen

#### Daten transformieren

In [ ]:
def convert_date(df, column_name):
    df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
    return df

In [ ]:
convert_date(df_fahrer, "Geburtsdatum")
convert_date(df_rennen, "Datum")

,Rennen_ID,Saison,GP_Name,Datum,Runde
0,2023_1,2023,Bahrain Grand Prix,2023-03-05,1
1,2023_2,2023,Saudi Arabian Grand Prix,2023-03-19,2
2,2023_3,2023,Australian Grand Prix,2023-04-02,3
3,2023_4,2023,Azerbaijan Grand Prix,2023-04-30,4
4,2023_5,2023,Miami Grand Prix,2023-05-07,5
...,...,...,...,...,...
65,2025_20,2025,Mexico City Grand Prix,2025-10-26,20
66,2025_21,2025,São Paulo Grand Prix,2025-11-09,21
67,2025_22,2025,Las Vegas Grand Prix,2025-11-23,22
68,2025_23,2025,Qatar Grand Prix,2025-11-30,23


#### Zeiten transformieren

In [ ]:
def convert_qualifying_times(df):
    def convert_to_milliseconds(time_str):
        if time_str == "\\N" or pd.isna(time_str):
            return pd.NA
        try:
            minutes, rest = time_str.split(":")
            seconds, milliseconds = rest.split(".")
            total_ms = (
                int(minutes) * 60 * 1000 +
                int(seconds) * 1000 +
                int(milliseconds)
            )
            return total_ms
        except ValueError:
            return pd.NA

    for col in ["Q1", "Q2", "Q3"]:
        if col in df.columns:
            df[col] = df[col].apply(convert_to_milliseconds).astype("Int64")

    return df

In [ ]:
convert_qualifying_times(df_quali)

,Saison,Runde,Fahrer_ID,Startplatz,Q1,Q2,Q3
0,2023,1,max_verstappen,1,91295,90503,89708
1,2023,1,perez,2,91479,90746,89846
2,2023,1,leclerc,3,91094,90282,90000
3,2023,1,sainz,4,90993,90515,90154
4,2023,1,alonso,5,91158,90645,90336
...,...,...,...,...,...,...,...
1114,2025,10,bortoleto,16,72385,<NA>,<NA>
1115,2025,10,sainz,17,72398,<NA>,<NA>
1116,2025,10,stroll,18,72517,<NA>,<NA>
1117,2025,10,lawson,19,72525,<NA>,<NA>


In [ ]:
df_fahrer.info()
df_rennen.info()
df_quali.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Fahrer_ID     68 non-null     object        
 1   Name          68 non-null     object        
 2   Nationalität  68 non-null     object        
 3   Geburtsdatum  68 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 2.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Rennen_ID  70 non-null     object        
 1   Saison     70 non-null     int64         
 2   GP_Name    70 non-null     object        
 3   Datum      70 non-null     datetime64[ns]
 4   Runde      70 non-null     int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 2.9+ KB
<class 'pandas.core.frame.DataFrame'>
R

In [ ]:
df_fahrer = df_fahrer.drop_duplicates(subset="Fahrer_ID")
df_konstrukteure = df_konstrukteure.drop_duplicates(subset="Konstrukteur_ID")


### Laden in Datenbank

In [ ]:
con.execute("INSERT INTO Fahrer SELECT * FROM df_fahrer")
con.execute("INSERT INTO Konstrukteur SELECT * FROM df_konstrukteure")
con.execute("INSERT INTO Rennen SELECT * FROM df_rennen")
con.execute("INSERT INTO Qualifikation SELECT Fahrer_ID, Runde, Q1, Q2, Q3, Startplatz, Saison FROM df_quali")
con.execute("INSERT INTO Ergebnis SELECT * FROM df_ergebnisse")


In [ ]:
con.close()